In [8]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

Catch_left_slope = 0
Catch_right_slope  = 0
Catch_left  = [0, 0, 0]
Catch_right  = [0, 0, 0]
def reset_globals():
    """
    Clear your globals before using new outputs
    """
    print("RESETING GLOBAL VALUES")
    global Catch_left_slope 
    global Catch_right_slope 
    global Catch_left   # X1, Y1, X2
    global Catch_right  
    Catch_left_slope  = 0
    Catch_right_slope  = 0
    Catch_left= [0, 0, 0]  # X1, Y1, X2
    Catch_right = [0, 0, 0]
    
def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
   
    global Catch_left_slope 
    global Catch_right_slope
    global Catch_left
    global Catch_right
    
    # DECLARE VARIABLES
    cache_weight = 0.9
    right_ys = []
    right_xs = []
    right_slopes = []

    left_ys = []
    left_xs = []
    left_slopes = []
    
    midpoint = img.shape[1] / 2  # X/2
    bottom_of_image = img.shape[0]  # Y
    
    for line in lines:
        for x1,y1,x2,y2 in line:   
            slope, intercept = np.polyfit((x1,x2), (y1,y2), 1)
            # Seprate points, and slope, lines
            if .35 < np.absolute(slope) <= .85: # Take all position and negative slopes
                if slope > 0 and x1 > midpoint and x2 > midpoint:
                    right_ys.append(y1)
                    right_ys.append(y2)
                    right_xs.append(x1)
                    right_xs.append(x2)
                    right_slopes.append(slope)
                elif slope < 0 and x1 < midpoint and x2 < midpoint:
                    left_ys.append(y1)
                    left_ys.append(y2)
                    left_xs.append(x1)
                    left_xs.append(x2)
                    left_slopes.append(slope)
                    
    # DRAW RIGHT LANE LINE
    if right_ys:
        right_index = right_ys.index(min(right_ys))  #index of y min wil be index of x min
        right_x1 = right_xs[right_index]
        right_y1 = right_ys[right_index]
        right_slope = np.median(right_slopes)
        if Catch_right_slope != 0:
            right_slope = right_slope + (Catch_right_slope - right_slope) * cache_weight

        right_x2 = int(right_x1 + (bottom_of_image - right_y1) / right_slope)
        
        if Catch_right_slope != 0:
            right_x1 = int(right_x1 + (Catch_right[0] - right_x1) * cache_weight)
            right_y1 = int(right_y1 + (Catch_right[1] - right_y1) * cache_weight)
            right_x2 = int(right_x2 + (Catch_right[2] - right_x2) * cache_weight)

        Catch_right_slope = right_slope
        Catch_right = [right_x1, right_y1, right_x2]

        cv2.line(img, (right_x1, right_y1), (right_x2, bottom_of_image), color, thickness)
        
          # DRAW LEFT LANE LINE
    if left_ys:
        left_index = left_ys.index(min(left_ys))
        left_x1 = left_xs[left_index]
        left_y1 = left_ys[left_index]
        left_slope = np.median(left_slopes)
        if Catch_left_slope != 0:
            left_slope = left_slope + (Catch_left_slope - left_slope) * cache_weight

        left_x2 = int(left_x1 + (bottom_of_image - left_y1) / left_slope)

        if Catch_left_slope != 0:
            left_x1 = int(left_x1 + (Catch_left[0] - left_x1) * cache_weight)
            left_y1 = int(left_y1 + (Catch_left[1] - left_y1) * cache_weight)
            left_x2 = int(left_x2 + (Catch_left[2] - left_x2) * cache_weight)

        Catch_left_slope = left_slope
        Catch_left = [left_x1, left_y1, left_x2]

        cv2.line(img, (left_x1, left_y1), (left_x2, bottom_of_image), color, thickness)


In [9]:


def process_image(image):
    """
    Takes in an image and uses Canny Edge Detection and Hough Transformations
    to detect lines in the image and then draw the lane lines onto the original
    image.
    """
    # PARAMETERS
    imshape = image.shape
    kernel_size = 3
    sigma_x = 0
    low_canny_threshold = 25
    high_canny_threshold = low_canny_threshold * 3
    vertices = np.array([[(0,imshape[0]), (9*imshape[1]/20, 11*imshape[0]/18), (11*imshape[1]/20, 11*imshape[0]/18), (imshape[1],imshape[0])]], dtype=np.int32)
    ignore_mask_color = 255
    rho = 1
    theta = np.pi/180
    hough_threshold = 10
    min_line_len = 30
    max_line_gap = 60
    α = 0.8
    β = 1.
    λ = 0.

    # GRAYSCALE
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # GAUSSIAN BLUR
    blur = cv2.GaussianBlur(gray, (kernel_size, kernel_size), sigma_x)

    # CANNY EDGES
    edges = cv2.Canny(blur, low_canny_threshold, high_canny_threshold)

    # REGION MASK
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked = cv2.bitwise_and(edges, mask)

    # HOUGH TRANSFORM
    lines = cv2.HoughLinesP(masked, rho, theta, hough_threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    hough_image = np.zeros((*masked.shape, 3), dtype=np.uint8)
    draw_lines(hough_image, lines)

    # WEIGHTED IMAGE
    processed = cv2.addWeighted(image, α, hough_image, β, λ)

    return processed


In [10]:
reset_globals()
white_output = 'white.mp4'
clip1 = VideoFileClip("challenge.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)
#Play video inline

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

RESETING GLOBAL VALUES
[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [00:18<00:00, 13.67it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 19.7 s
